In [11]:
# Deep Research CrewAI Agent with Local Ollama and DuckDuckGo Search
# Install required packages first:
# !pip install crewai crewai-tools python-dotenv requests beautifulsoup4 duckduckgo-search

import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, FileReadTool
from crewai.llm import LLM
import requests
from bs4 import BeautifulSoup
import json

# Load environment variables
load_dotenv()

print("Setting up Deep Research CrewAI Agent with Local Ollama and DuckDuckGo...")


Setting up Deep Research CrewAI Agent with Local Ollama and DuckDuckGo...


In [12]:
# Local Ollama Configuration
class OllamaResearchConfig:
    def __init__(self):
        # Set your Ollama configuration
        self.base_url = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
        self.model_name = os.getenv("OLLAMA_MODEL", "llama3.2")  # Change to your preferred model
        self.timeout = int(os.getenv("OLLAMA_TIMEOUT", "300"))  # 5 minutes timeout
        
    def get_llm(self):
        """Configure Ollama LLM for CrewAI"""
        return LLM(
            model=f"ollama/{self.model_name}",
            base_url=self.base_url,
            temperature=0.1,  # Low temperature for more focused research
            max_tokens=5000,
            timeout=self.timeout,
            # Ollama-specific parameters
            top_p=0.9,
            top_k=50,
            repeat_penalty=1.1
        )

config = OllamaResearchConfig()
local_llm = config.get_llm()

print(f"Ollama configuration loaded successfully! Using model: {config.model_name}")
print(f"Ollama server: {config.base_url}")


Ollama configuration loaded successfully! Using model: llama3.2
Ollama server: http://localhost:11434


In [13]:
# Custom Research Tools
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

class WebResearchInput(BaseModel):
    query: str = Field(..., description="The search query to research")
    max_results: int = Field(default=10, description="Maximum number of results to return")

class WebResearchTool(BaseTool):
    name: str = "web_research_tool"
    description: str = "Performs comprehensive web research using DuckDuckGo search"
    args_schema: Type[BaseModel] = WebResearchInput
    
    def _run(self, query: str, max_results: int = 10) -> str:
        """Perform web search using DuckDuckGo and return formatted results"""
        try:
            from duckduckgo_search import DDGS
            
            results = []
            
            # Perform DuckDuckGo search
            with DDGS() as ddgs:
                search_results = ddgs.text(
                    keywords=query,
                    max_results=min(max_results, 20),  # DuckDuckGo typically returns up to 20 results
                    region='us-en',
                    safesearch='moderate',
                    timelimit=None
                )
                
                # Process search results
                for i, result in enumerate(search_results, 1):
                    results.append(f"""
Result {i}:
Title: {result.get('title', 'N/A')}
URL: {result.get('href', 'N/A')}
Snippet: {result.get('body', 'N/A')}
---""")
            
            if not results:
                return f"No results found for query: '{query}'"
            
            return f"DuckDuckGo Web Search Results for '{query}':\n" + "\n".join(results)
        
        except ImportError:
            return "Error: duckduckgo-search package not installed. Please install it with: pip install duckduckgo-search"
        except Exception as e:
            return f"Error performing web research with DuckDuckGo: {str(e)}"

class ContentAnalysisInput(BaseModel):
    url: str = Field(..., description="URL to extract and analyze content from")

class ContentAnalysisTool(BaseTool):
    name: str = "content_analysis_tool"
    description: str = "Extracts and analyzes content from web pages"
    args_schema: Type[BaseModel] = ContentAnalysisInput
    
    def _run(self, url: str) -> str:
        """Extract and analyze content from a webpage"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract title
            title = soup.find('title')
            title_text = title.get_text(strip=True) if title else "No title found"
            
            # Extract main content
            # Try to find main content areas
            content_selectors = ['main', 'article', '.content', '.post-content', '.entry-content', 'body']
            content = ""
            
            for selector in content_selectors:
                elements = soup.select(selector)
                if elements:
                    content = elements[0].get_text(separator=' ', strip=True)
                    break
            
            if not content:
                content = soup.get_text(separator=' ', strip=True)
            
            # Limit content length for processing
            content = content[:5000] + "..." if len(content) > 5000 else content
            
            return f"""
URL: {url}
Title: {title_text}
Content Analysis:
{content}
"""
        
        except Exception as e:
            return f"Error analyzing content from {url}: {str(e)}"

# Initialize tools
web_research_tool = WebResearchTool()
content_analysis_tool = ContentAnalysisTool()

print("Custom research tools initialized successfully!")


Custom research tools initialized successfully!


In [14]:
# Deep Research Agent Definition
class DeepResearchAgent:
    def __init__(self, llm):
        self.llm = llm
        
    def create_research_agent(self):
        """Create a specialized research agent"""
        return Agent(
            role='Senior Research Analyst',
            goal='Conduct comprehensive and thorough research on any given topic, providing detailed insights, analysis, and evidence-based conclusions',
            backstory="""You are a world-class research analyst with expertise across multiple domains. 
            You have a methodical approach to research, always seeking multiple sources, cross-referencing information, 
            and providing balanced, well-supported conclusions. You excel at identifying key trends, patterns, 
            and implications in your research findings. Your research is always thorough, accurate, and actionable.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm,
            tools=[web_research_tool, content_analysis_tool],
            max_iter=5,
            max_execution_time=1800  # 30 minutes max
        )
    
    def create_synthesis_agent(self):
        """Create an agent specialized in synthesizing research findings"""
        return Agent(
            role='Research Synthesis Expert',
            goal='Synthesize complex research findings into clear, actionable insights and comprehensive reports',
            backstory="""You are an expert at taking vast amounts of research data and distilling it into 
            clear, actionable insights. You excel at identifying patterns, themes, and connections across 
            different sources of information. Your reports are well-structured, easy to understand, 
            and provide practical recommendations based on the research findings.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm,
            tools=[],
            max_iter=3
        )

# Initialize the research system
research_system = DeepResearchAgent(local_llm)
research_agent = research_system.create_research_agent()
synthesis_agent = research_system.create_synthesis_agent()

print("Research agents created successfully!")


Research agents created successfully!


In [15]:
# Research Tasks Definition
class ResearchTasks:
    def __init__(self):
        pass
    
    def create_deep_research_task(self, topic: str, specific_aspects: str = ""):
        """Create a comprehensive research task"""
        aspects_instruction = f"\nFocus particularly on: {specific_aspects}" if specific_aspects else ""
        
        return Task(
            description=f"""
            Conduct a comprehensive and deep research on the topic: "{topic}"{aspects_instruction}
            
            Your research should include:
            1. Multiple search queries to gather diverse perspectives and sources
            2. Analysis of credible sources, recent developments, and expert opinions
            3. Identification of key trends, patterns, and emerging insights
            4. Cross-referencing information to ensure accuracy and completeness
            5. Exploration of different viewpoints and potential controversies
            6. Collection of relevant statistics, data, and factual information
            7. Identification of key stakeholders and industry leaders
            
            Use your web research tools extensively to gather information from various sources.
            Aim for depth and breadth in your research coverage.
            """,
            expected_output="""
            A comprehensive research report containing:
            - Executive summary of key findings
            - Detailed analysis of the topic from multiple angles
            - Key statistics and data points with sources
            - Recent developments and trends
            - Different perspectives and viewpoints
            - Potential implications and future outlook
            - List of credible sources used
            """,
            agent=research_agent
        )
    
    def create_synthesis_task(self, topic: str):
        """Create a task to synthesize research findings"""
        return Task(
            description=f"""
            Synthesize the comprehensive research findings about "{topic}" into a clear, 
            well-structured, and actionable report.
            
            Your synthesis should:
            1. Organize the information logically and coherently
            2. Identify the most important insights and themes
            3. Present balanced viewpoints and acknowledge uncertainties
            4. Provide actionable recommendations where appropriate
            5. Highlight areas that may need further investigation
            6. Create a clear narrative that connects all the research findings
            
            Focus on making complex information accessible and useful.
            """,
            expected_output="""
            A polished final research report with:
            - Clear executive summary
            - Well-organized main findings
            - Key insights and implications  
            - Actionable recommendations
            - Areas for further research
            - Professional formatting and structure
            """,
            agent=synthesis_agent
        )

# Initialize tasks handler
tasks_handler = ResearchTasks()

print("Research tasks framework created successfully!")


Research tasks framework created successfully!


In [16]:
# Deep Research Crew Orchestration
class DeepResearchCrew:
    def __init__(self):
        self.research_system = research_system
        self.tasks_handler = tasks_handler
    
    def conduct_research(self, topic: str, specific_aspects: str = "", save_to_file: bool = True):
        """
        Conduct comprehensive research on a given topic
        
        Args:
            topic (str): The main topic to research
            specific_aspects (str): Specific aspects to focus on (optional)
            save_to_file (bool): Whether to save results to a file
        
        Returns:
            str: The final research report
        """
        print(f"🔍 Starting deep research on: {topic}")
        if specific_aspects:
            print(f"📋 Focus areas: {specific_aspects}")
        
        # Create tasks
        research_task = self.tasks_handler.create_deep_research_task(topic, specific_aspects)
        synthesis_task = self.tasks_handler.create_synthesis_task(topic)
        
        # Create crew
        crew = Crew(
            agents=[research_agent, synthesis_agent],
            tasks=[research_task, synthesis_task],
            process=Process.sequential,
            verbose=True,
            memory=False  # Disable memory to avoid configuration complexity
        )
        
        # Execute research
        try:
            print("🚀 Executing research crew...")
            result = crew.kickoff()
            
            # Get the final output
            final_report = str(result)
            
            # Save to file if requested
            if save_to_file:
                filename = f"research_report_{topic.replace(' ', '_').replace('/', '_')}.md"
                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(f"# Deep Research Report: {topic}\n\n")
                    f.write(f"Generated by CrewAI Deep Research Agent\n")
                    f.write(f"Date: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
                    f.write("---\n\n")
                    f.write(final_report)
                
                print(f"📄 Research report saved to: {filename}")
            
            return final_report
            
        except Exception as e:
            error_msg = f"❌ Error during research execution: {str(e)}"
            print(error_msg)
            return error_msg

# Initialize the research crew
research_crew = DeepResearchCrew()

print("🎯 Deep Research Crew is ready for action!")


🎯 Deep Research Crew is ready for action!


In [17]:
# Example Usage and Configuration

# Step 1: Set up your environment variables (optional - defaults provided)
print("📋 Configuration Setup:")
print("1. Create a .env file in your project directory with (optional):")
print("""
OLLAMA_BASE_URL=http://localhost:11434
OLLAMA_MODEL=llama3.2
OLLAMA_TIMEOUT=300
""")

print("\n2. Install and setup Ollama:")
print("   - Download from https://ollama.ai")
print("   - Run: ollama pull llama3.2")
print("   - Start server: ollama serve")

print("\n3. Install required packages:")
print("pip install crewai crewai-tools python-dotenv requests beautifulsoup4 duckduckgo-search")

print("\n" + "="*60)
print("🚀 READY TO USE - Example Research Topics:")
print("="*60)

# Example usage function
def run_example_research():
    """Run example research on AI trends"""
    topic = "Artificial Intelligence trends in 2024"
    specific_aspects = "machine learning, generative AI, enterprise adoption, ethical considerations"
    
    result = research_crew.conduct_research(
        topic=topic,
        specific_aspects=specific_aspects,
        save_to_file=True
    )
    
    print("✅ Research completed!")
    return result

# Uncomment the line below to run example research
# example_result = run_example_research()


📋 Configuration Setup:
1. Create a .env file in your project directory with (optional):

OLLAMA_BASE_URL=http://localhost:11434
OLLAMA_MODEL=llama3.2
OLLAMA_TIMEOUT=300


2. Install and setup Ollama:
   - Download from https://ollama.ai
   - Run: ollama pull llama3.2
   - Start server: ollama serve

3. Install required packages:
pip install crewai crewai-tools python-dotenv requests beautifulsoup4 duckduckgo-search

🚀 READY TO USE - Example Research Topics:


# Ollama Setup Instructions

## Prerequisites
1. **Install Ollama**: Download and install from [https://ollama.ai](https://ollama.ai)
2. **Pull a Llama model**: Run in terminal:
   ```bash
   ollama pull llama3.2
   # or for larger models:
   ollama pull llama3.2:13b
   ollama pull llama3.2:70b
   ```
3. **Start Ollama server**: 
   ```bash
   ollama serve
   ```

## Environment Configuration
Create a `.env` file with these variables (optional, defaults provided):
```
OLLAMA_BASE_URL=http://localhost:11434
OLLAMA_MODEL=llama3.2
OLLAMA_TIMEOUT=300
```

## Required Packages
Install the required Python packages:
```bash
pip install crewai crewai-tools python-dotenv requests beautifulsoup4 duckduckgo-search
```

## Search Engine
- **DuckDuckGo**: No API key required, privacy-focused search
- Free and unlimited searches
- No registration or authentication needed

## Available Models
- `llama3.2` (3B) - Fast, good for basic tasks
- `llama3.2:7b` - Balanced performance and quality
- `llama3.2:13b` - Better quality, slower
- `llama3.2:70b` - Best quality, requires significant RAM
- `codellama` - Specialized for code generation
- `mistral` - Alternative model option

## Troubleshooting
- Ensure Ollama is running: `ollama list`
- Check server status: `curl http://localhost:11434/api/tags`
- Monitor resource usage during execution

In [18]:
# 🎯 CONDUCT YOUR RESEARCH HERE

def conduct_custom_research():
    """Interactive function to conduct research on your chosen topic"""
    
    # Customize these variables for your research
    research_topic = "Your research topic here"  # Change this to your topic
    focus_areas = ""  # Optional: specific aspects to focus on
    
    print(f"🔍 Conducting research on: {research_topic}")
    
    if focus_areas:
        print(f"📋 Focus areas: {focus_areas}")
    
    # Execute the research
    try:
        result = research_crew.conduct_research(
            topic=research_topic,
            specific_aspects=focus_areas,
            save_to_file=True
        )
        
        print("✅ Research completed successfully!")
        print("📄 Check your project directory for the generated research report.")
        
        return result
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Make sure Ollama is running and you have pulled the required model")
        print("   Run: ollama serve")
        print("   Run: ollama pull llama3.2")
        return None

# Instructions for use:
print("🎯 TO CONDUCT RESEARCH:")
print("1. Edit the 'research_topic' variable above with your topic")
print("2. Optionally add specific focus areas")
print("3. Run: conduct_custom_research()")
print("\n" + "="*60)
print("📚 Example Topics:")
print("- 'Climate change impact on agriculture'")
print("- 'Blockchain technology in healthcare'") 
print("- 'Remote work productivity strategies'")
print("- 'Sustainable energy solutions 2024'")
print("="*60)

# Uncomment the line below and customize the topic above to run your research
# result = conduct_custom_research()


🎯 TO CONDUCT RESEARCH:
1. Edit the 'research_topic' variable above with your topic
2. Optionally add specific focus areas
3. Run: conduct_custom_research()

📚 Example Topics:
- 'Climate change impact on agriculture'
- 'Blockchain technology in healthcare'
- 'Remote work productivity strategies'
- 'Sustainable energy solutions 2024'


In [19]:
# Example research topics:
result = research_crew.conduct_research(
    topic="Artificial Intelligence trends in 2024",
    specific_aspects="machine learning, generative AI, enterprise adoption",
    save_to_file=True
)

2025-06-28 22:45:02,243 - 4196 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


🔍 Starting deep research on: Artificial Intelligence trends in 2024
📋 Focus areas: machine learning, generative AI, enterprise adoption
🚀 Executing research crew...
# Agent: Senior Research Analyst
## Task: 
            Conduct a comprehensive and deep research on the topic: "Artificial Intelligence trends in 2024"
Focus particularly on: machine learning, generative AI, enterprise adoption
            
            Your research should include:
            1. Multiple search queries to gather diverse perspectives and sources
            2. Analysis of credible sources, recent developments, and expert opinions
            3. Identification of key trends, patterns, and emerging insights
            4. Cross-referencing information to ensure accuracy and completeness
            5. Exploration of different viewpoints and potential controversies
            6. Collection of relevant statistics, data, and factual information
            7. Identification of key stakeholders and industry lead